In [1]:
#두 문장간 유사도 측정(TF-IDF)
from pandas import DataFrame #Series, 

data = {}
data['gd']=[
  'MEDICAL EYE WASH SOLUTION 500ML KOBAYASHI',
  'DVF DRESSVINCE JACKET',
  '(MP3)JBL PULSE 2 PORTABLE SPLASHPROOF BLUETOOTH SPEAKER ONE SIZE',
  'ARTICLES OF DAILY USE A GLASS OF WINE',
  'MEDICAL EYE WASH SOLUTION 500ML KOBAYASHI AAA',  
  'CHAMPSSPORTS CHAMPSSPORTS NIKE AIR FLEECE PANTS'
]
#data['index']=[0,1,2,3,4]

data = DataFrame(data)
print(data)


                                                  gd
0          MEDICAL EYE WASH SOLUTION 500ML KOBAYASHI
1                              DVF DRESSVINCE JACKET
2  (MP3)JBL PULSE 2 PORTABLE SPLASHPROOF BLUETOOT...
3              ARTICLES OF DAILY USE A GLASS OF WINE
4      MEDICAL EYE WASH SOLUTION 500ML KOBAYASHI AAA
5    CHAMPSSPORTS CHAMPSSPORTS NIKE AIR FLEECE PANTS


In [9]:
def cal_tfidf(data):
    from sklearn.feature_extraction.text import TfidfVectorizer
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(data['gd'])
    # overview에 대해서 tf-idf 수행
    print('tfidf_matrix:', tfidf_matrix.shape)
    #print(tfidf_matrix)

    from sklearn.metrics.pairwise import linear_kernel
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    #print(cosine_sim)

    #중복제거?
    import pandas as pd
    indices = pd.Series(data.index, index=data['gd']).drop_duplicates()
    #print(indices.head())

    #idx = indices['MEDICAL EYE WASH SOLUTION 500ML KOBAYASHI']
    #print(idx)

    return indices, cosine_sim

indices, cosine_sim = cal_tfidf(data)

tfidf_matrix: (6, 28)


In [0]:
def get_recommendations(title, indices, cosine_sim=cosine_sim):
    loc = indices.get(title)
    print('word loc:', loc)

    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    print('sim_scores:', sim_scores)

    # 가장 유사한 3개의 문장을 받아옵니다.
    sim_scores = sim_scores[1:1+3]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return data['gd'].iloc[movie_indices]

In [12]:
#새로운 단어/문장 추가
add_data = DataFrame({'gd':['MEDICAL EYE WASH ']})
new_data = data.append(add_data, ignore_index = True)
#print(new_data)

#유사도 재계산
indices, cosine_sim  = cal_tfidf(new_data)

#유사목록 출력
get_recommendations('MEDICAL EYE WASH ', indices, cosine_sim)

tfidf_matrix: (7, 28)
word loc: 6
sim_scores: [(6, 1.0), (0, 0.649749587643745), (4, 0.5744082542553888), (1, 0.0), (2, 0.0), (3, 0.0), (5, 0.0)]


0        MEDICAL EYE WASH SOLUTION 500ML KOBAYASHI
4    MEDICAL EYE WASH SOLUTION 500ML KOBAYASHI AAA
1                            DVF DRESSVINCE JACKET
Name: gd, dtype: object